## Step 1: Creating Token

In [1]:
with open('the-verdict.txt', 'r', encoding = 'utf-8') as f:
    raw_text = f.read()
print("Total number of character: ", len(raw_text) )
print(raw_text[:100])

Total number of character:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g


In [4]:
import re
text = "hello, world. This, is a test"
result = re.split(r'(\s)', text)
result

['hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test']

In [7]:
result = re.split(r'([,.]|\s)', text)
result

['hello',
 ',',
 '',
 ' ',
 'world',
 '.',
 '',
 ' ',
 'This',
 ',',
 '',
 ' ',
 'is',
 ' ',
 'a',
 ' ',
 'test']

In [8]:
# let's remove white space
result = [item for item in result if item.strip()]
result

['hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test']

Removing white space saves memory, however, having spaces are useful in some applications

Incorporate Punctuation marks into tokens

In [15]:
text = "Hello, world! Is this-- a test?"
result = re.split(r'([.,!()\/:;?_]|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
result

['Hello', ',', 'world', '!', 'Is', 'this', '--', 'a', 'test', '?']

In [17]:
# Strip whitespace form each item
result = [item for item in result if item.strip()]
result

['Hello', ',', 'world', '!', 'Is', 'this', '--', 'a', 'test', '?']

Let's apply the above tokenization scheme to the raw text

In [22]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
len(preprocessed)

4690

In [23]:
preprocessed[:10]

['I',
 'HAD',
 'always',
 'thought',
 'Jack',
 'Gisburn',
 'rather',
 'a',
 'cheap',
 'genius']

## Step 2: Token > Token ID

In [31]:
all_words = sorted(list(set(preprocessed)))
vocab_size = len(all_words)
vocab_size

1130

In [33]:
vocab = {token:integer for integer, token in enumerate(all_words)}

In [37]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i == 10:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)


For the decoder we need to convert IDs > Tokens
(Inversion of the above)

In [53]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[i] for i in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [51]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last painted, you know,"
          Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
tokens = tokenizer.decode(ids)

In [52]:
tokens

'" It\' s the last painted, you know," Mrs. Gisburn said with pardonable pride.'

In [56]:
# Ading special context tokens to handle unknown words 
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer, token in enumerate(all_tokens)}

In [57]:
len(vocab.items())

1132

In [60]:
# let's print the last 5 vocab

for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [61]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]
        ids = [self.str_to_int[i] for i in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [63]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))
text

'Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.'

In [64]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [65]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'